In [ ]:
# --- Étape 2 : Charger les données ---
from src.pipeline import Pipeline
pipe = Pipeline()
df_reviews = pipe.get_every_reviews()

In [ ]:
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px
from textblob import TextBlob

# Charger les stopwords français
STOP_WORDS_URL = "https://raw.githubusercontent.com/stopwords-iso/stopwords-fr/master/stopwords-fr.txt"
stop_words = pd.read_csv(STOP_WORDS_URL, header=None)[0].tolist()

# Initialiser les outils de NLP
lemmatizer = WordNetLemmatizer()

# --- Étape 1 : Prétraitement du texte ---
def clean_text(text):
    """Nettoie un texte en supprimant les caractères spéciaux, ponctuation, etc."""
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Supprimer les chiffres
    text = text.translate(str.maketrans('', '', string.punctuation))  # Supprimer la ponctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Supprimer les espaces multiples
    text = re.sub(r'\b\w\b', '', text)  # Supprimer les caractères seuls
    return text

def preprocess(review):
    """Nettoie, tokenize, et lemmatise un texte tout en supprimant les stopwords."""
    review = review.lower()
    tokens = word_tokenize(review)
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

# --- Étape 2 : Charger les données ---
# pipe = Pipeline()
# df_reviews = pipe.get_every_reviews()
df_reviews["cleaned_review"] = df_reviews["review"].apply(preprocess)

# --- Étape 3 : Analyse des sentiments ---
df_reviews["sentiment"] = df_reviews["review"].apply(lambda x: TextBlob(x).sentiment.polarity)
df_reviews["sentiment_label"] = df_reviews["sentiment"].apply(lambda x: "positive" if x >= 0 else "negative")

# --- Étape 4 : Clustering des restaurants ---
# Charger les données des restaurants
restaurants = pipe.get_all_restaurants()
df_restaurants = pd.DataFrame([{
    "id_restaurant": r.id_restaurant,
    "nom": r.nom,
    "latitude": r.latitude,
    "longitude": r.longitude,
    "rank": r.rank,
    "prix_min": r.prix_min,
    "prix_max": r.prix_max,
    "etoiles_michelin": r.etoiles_michelin,
    "note_globale": r.note_globale,
    "qualite_prix_note": r.qualite_prix_note,
    "cuisine_note": r.cuisine_note,
    "service_note": r.service_note,
    "ambiance_note": r.ambiance_note,
    "cuisines": r.cuisines
} for r in restaurants if r.scrapped == 1])

# Joindre les avis nettoyés avec les informations des restaurants
df = pd.merge(df_reviews, df_restaurants, left_on="restaurant_id", right_on="id_restaurant", how="inner")
print(df.head())


   restaurant_id  user_id  review_id  \
0              1        1          1   
1              1        2          2   
2              1        3          3   
3              1        4          4   
4              1        5          5   

                                               title     user_profile  \
0                                  Très belle soirée           SetC77   
1  Vive la bonne cuisine dans une ambiance conviv...   H3293ZGsylviel   
2                                          Sans plus  marieno_lleb739   
3                                      Bon et joyeux          Vymsbmm   
4  Bon restaurant et endroit pour se retrouver en...       Youliic974   

  date_review  rating type_visit  num_contributions  \
0  2024-12-16     5.0    friends                 67   
1  2024-11-26     5.0    friends                  2   
2  2024-11-23     3.0    friends                 96   
3  2024-11-14     5.0    friends                225   
4  2024-11-01     4.0    friends             

In [53]:
df["cleaned_review"]

0        moment accueil plat ambiance passer moment ami...
1        goûteux l'ambiance bonne humeur moment convivi...
2        serveurs serveuses sympas onglet tendre terrin...
3        excellente soirée petit restaurant écart côté ...
4        super moment amis plat délicieux conseille for...
                               ...                        
20656    petit restaurant quartier service impeccable s...
20657    meilleurs coréens jamais tester service agréab...
20658    eme visite café ambiance cosy plat délicieux m...
20659    amis lyonnais découvrir restaurant coréen d'ab...
20660    accueil chaleureux serveur souriant plat jolim...
Name: cleaned_review, Length: 20661, dtype: object

In [54]:
# --- Étape 5 : Vectorisation des avis avec TF-IDF ---
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_tfidf = tfidf_vectorizer.fit_transform(df["cleaned_review"])

# Ajouter les vecteurs TF-IDF agrégés par restaurant
df["tfidf_vector"] = list(X_tfidf.toarray())
aggregated_tfidf = (
    df.groupby("restaurant_id")["tfidf_vector"]
    .apply(lambda x: np.mean(np.vstack(x), axis=0))  # Moyenne des vecteurs TF-IDF
    .reset_index()
)
aggregated_tfidf.columns = ["id_restaurant", "tfidf_vector"]
df_restaurants = df_restaurants.merge(aggregated_tfidf, on="id_restaurant", how="left")

# --- Étape 6 : Clustering avec KMeans ---
features = ["prix_min", "prix_max", "note_globale", "qualite_prix_note", "cuisine_note", "service_note", "ambiance_note"]
restaurant_features = pd.concat([df_restaurants[features], pd.DataFrame(aggregated_tfidf["tfidf_vector"].to_list())], axis=1)
restaurant_features.columns = restaurant_features.columns.astype(str)


In [55]:
tfidf_vectorizer.get_feature_names_out()

array(['00', '000', '02', ..., 'œuvre', 'œuvrer', 'œuvres'], dtype=object)

In [56]:
# --- Étape 8 : Recommandation basée sur mot-clé ---
def find_restaurant_by_keyword(keyword, vectorizer, restaurant_features):
    """Trouver le restaurant le plus pertinent en fonction d'un mot-clé."""
    keyword_vector = vectorizer.transform([keyword])
    restaurant_vectors = np.vstack(restaurant_features["tfidf_vector"])
    similarities = cosine_similarity(keyword_vector, restaurant_vectors).flatten()
    best_match_idx = np.argmax(similarities)
    best_match = restaurant_features.iloc[best_match_idx]
    return {
        "nom": best_match["nom"],
        "note_globale": best_match["note_globale"],
        "prix_min": best_match["prix_min"],
        "prix_max": best_match["prix_max"],
        "cuisines": best_match["cuisines"],
        "similarity_score": similarities[best_match_idx]
    }

# Exemple de recherche
keyword = "italienne"
best_match = find_restaurant_by_keyword(keyword, tfidf_vectorizer, df_restaurants)
print(f"Restaurant recommandé pour le mot-clé '{keyword}': {best_match}")

Restaurant recommandé pour le mot-clé 'italienne': {'nom': 'Carmelo', 'note_globale': 4.0, 'prix_min': 9.0, 'prix_max': 25.0, 'cuisines': 'Italienne, Toscane, Romana, Latium, Sicilienne, Ligurienne, Italie du Nord, Italie du Centre, Italie du Sud', 'similarity_score': 0.13367823167586754}


In [57]:
# recode in str the columns
restaurant_features.columns = restaurant_features.columns.astype(str)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(restaurant_features)

# Réduction de dimensions
pca = PCA(n_components=3)
# remove nan values
features_scaled = np.nan_to_num(features_scaled)
features_3d = pca.fit_transform(features_scaled)

# Appliquer KMeans
kmeans = KMeans(n_clusters=5, random_state=42)
df_restaurants["cluster"] = kmeans.fit_predict(features_scaled)

# --- Étape 7 : Visualisation des clusters ---
visual_df = pd.DataFrame(features_3d, columns=["PCA1", "PCA2", "PCA3"])
visual_df["cluster"] = df_restaurants["cluster"]
visual_df["restaurant_name"] = df_restaurants["nom"]

fig = px.scatter_3d(
    visual_df, x="PCA1", y="PCA2", z="PCA3", color="cluster",
    hover_data=["restaurant_name"], title="Clustering des restaurants"
)
fig.show()




In [59]:
keyword = "grillades"
best_match = find_restaurant_by_keyword(keyword, tfidf_vectorizer, df_restaurants)
print(f"Restaurant recommandé pour le mot-clé '{keyword}': {best_match}")

Restaurant recommandé pour le mot-clé 'grillades': {'nom': 'Mama Restaurant Lyon', 'note_globale': 4.0, 'prix_min': 7.0, 'prix_max': 47.0, 'cuisines': 'Française, Bars-restaurants', 'similarity_score': 0.0024091894144973895}
